In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bestsellers.csv to bestsellers.csv


Se visualizan los tipos de datos que componen el propio dataset

In [ ]:
import pandas as pd
import io
import numpy as np

bestsellers = pd.read_csv(io.BytesIO(uploaded['bestsellers.csv']))
bestsellers.head(5)

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [ ]:
bestsellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


In [ ]:
bestsellers.describe()

,User Rating,Reviews,Price,Year
count,550.000000,550.000000,550.000000,550.000000
mean,4.618364,11953.281818,13.100000,2014.000000
std,0.226980,11731.132017,10.842262,3.165156
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4058.000000,7.000000,2011.000000
50%,4.700000,8580.000000,11.000000,2014.000000
75%,4.800000,17253.250000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


Histograma de libros con las diferentes puntuaciones proporcionadas por los usuarios

In [ ]:
import plotly.express as px

df = bestsellers.copy()
fig = px.histogram(df, x="User Rating")
fig.update_layout(bargap=0.2, paper_bgcolor="#ffb833")
fig.show()

Gráfico de quesito de la distribución de los diferentes ratios de usuario.

In [ ]:
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.offline as py


labels=df['User Rating'].value_counts().index
values=df['User Rating'].value_counts().values
go.Figure(data=go.Pie(labels=labels, 
                   values=values, 
                   title='Rating', 
                   hoverinfo='value+percent', 
                   textinfo='percent',
                   textposition='inside',
                   hole=0.5,
                   showlegend=True,
                   marker=dict(colors=plt.cm.Set2(np.linspace(0, 1, len(np.unique(labels)))),
                               line=dict(color='#000000',
                                         width=4),
                              )
                  )
)

Ficción vs. No ficción

In [ ]:
fig = px.histogram(y=df['Genre'].value_counts().reset_index()['Genre'], x=df['Genre'].value_counts().reset_index()['index'], labels={'x':'libros', 'y':'libros'})
fig.update_layout(bargap=0.2, paper_bgcolor="#ffb833")
fig.show()

In [ ]:
labels=df['Genre'].value_counts().index
values=df['Genre'].value_counts().values
go.Figure(data=go.Pie(labels=labels, 
                   values=values, 
                   title='Género', 
                   hoverinfo='value+percent', 
                   textinfo='percent',
                   textposition='inside',
                   hole=0.5,
                   showlegend=True,
                   marker=dict(colors=plt.cm.Set2(np.linspace(0, 1, len(np.unique(labels)))),
                               line=dict(color='#000000',
                                         width=4),
                              )
                  )
)

Rating por género

In [ ]:
ficcion = df[df["Genre"] == "Fiction"]["User Rating"].value_counts().reset_index()
no_ficcion = df[df["Genre"] == "Non Fiction"]["User Rating"].value_counts().reset_index()


ficcion = ficcion.rename(columns = {"User Rating" : "total", "index" : "User Rating"})
no_ficcion = no_ficcion.rename(columns = {"User Rating" : "total", "index" : "User Rating"})



non_fiction_bar = go.Bar(
                    y=no_ficcion["total"],
                    x=no_ficcion["User Rating"],
                    name="No Ficción", 
                    marker=dict(color = '#3fe8bc',
                             line=dict(color='rgb(0,0,0)',width=1.5)))

fiction_bar = go.Bar(
                    y=ficcion["total"],
                    x=ficcion["User Rating"],
                    name="Ficción", 
                    marker=dict(color = '#BF40BF',
                             line=dict(color='rgb(0,0,0)',width=1.5)))

layout = go.Layout(template= "none",title = 'Rating por género' , yaxis = dict(title = 'Total'), xaxis = dict(title = 'Rating'))
fig = go.Figure(data = [non_fiction_bar, fiction_bar], layout = layout)
fig.show()

Best sellers a lo largo de los años

In [ ]:
ficcion = df[df["Genre"] == "Fiction"]["Year"].value_counts().reset_index()
ficcion = ficcion.rename(columns = {"Year" : "total", "index" : "Year"})
ficcion['percent'] = ficcion['total'].apply(lambda x : 100*x/sum(ficcion['total']))
ficcion = ficcion.sort_values("Year")

no_ficcion = df[df["Genre"] == "Non Fiction"]["Year"].value_counts().reset_index()
no_ficcion = no_ficcion.rename(columns = {"Year" : "total", "index" : "Year"})
no_ficcion['percent'] = no_ficcion['total'].apply(lambda x : 100*x/sum(no_ficcion['total']))
no_ficcion = no_ficcion.sort_values("Year")


non_fiction = go.Scatter(
                    y=no_ficcion["total"],
                    x=no_ficcion["Year"],
                    name="No Ficción", 
                    marker=dict(color = '#3fe8bc',
                             line=dict(color='rgb(0,0,0)',width=1.5)))

fiction = go.Scatter(
                    y=ficcion["total"],
                    x=ficcion["Year"],
                    name="Ficción", 
                    marker=dict(color = '#BF40BF',
                             line=dict(color='rgb(0,0,0)',width=1.5)))



layout = go.Layout(hovermode= 'closest', title = 'Bestsellers según el género', yaxis = dict(title = 'Total'), xaxis = dict(title = 'Años'),template= "none")
fig = go.Figure(data = [non_fiction, fiction], layout=layout)
fig.show()

¿Libros con diferentes ediciones?

In [ ]:
df['Name'].value_counts().reset_index()

,index,Name
0,Publication Manual of the American Psychologic...,10
1,StrengthsFinder 2.0,9
2,"Oh, the Places You'll Go!",8
3,The Very Hungry Caterpillar,7
4,The 7 Habits of Highly Effective People: Power...,7
...,...,...
346,Humans of New York : Stories,1
347,Howard Stern Comes Again,1
348,Homebody: A Guide to Creating Spaces You Never...,1
349,Have a Little Faith: A True Story,1


Se preserva únicamente una de las ediciones en el dataset, ya que, todas las ediciones comparten las mismas estadísticas, estando, por tanto, algunas filas "duplicadas".

In [ ]:
df_dedup = df.copy()

df_dedup.drop_duplicates(subset='Name', inplace=True)
df_dedup.shape

(351, 7)

In [ ]:
df_dedup = df.copy()
df_dedup.drop_duplicates(subset='Name', inplace=True)
df_orig = df['Name'].value_counts().reset_index()
df_orig = df_orig.rename(columns = {"Name" : "total", "index" : "Name"})
df_dedup = pd.merge(df_dedup, df_orig, on="Name")
df_dedup

,Name,Author,User Rating,Reviews,Price,Year,Genre,total
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction,1
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction,1
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction,1
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction,1
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction,1
...,...,...,...,...,...,...,...,...
346,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15,2012,Fiction,1
347,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11,2010,Non Fiction,1
348,Wonder,R. J. Palacio,4.8,21625,9,2013,Fiction,5
349,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction,1


Evolución de los libros por ratio a lo largo de los años, según su género.

In [ ]:
fig = px.scatter(df_dedup, x="Year", y='User Rating', color="Genre",hover_data=['Name'], size='total', facet_col="Genre")

fig.update_layout(template='simple_white', title='Ratio por género a lo largo de los años', yaxis_title='Ratio',
                  legend=dict(title='', itemclick='toggle', itemsizing='constant', traceorder='normal',
                  bgcolor='rgba(0,0,0,0)', x=1),
                  xaxis=dict(title='Género', showticklabels=True, ticks='outside', type='category')
                 )
fig.show()

Libros que más ediciones presentan.

In [ ]:
most_repeated = df_dedup.sort_values("total")[::-1][:20]

fig = go.Figure(go.Funnel(
    x=most_repeated["total"],
    y=most_repeated["Name"])
)

fig.show()

Los 20 libros con mejor ratio y su precio

In [ ]:
most_expensive = df_dedup.sort_values("Price")[::-1][:20]

In [ ]:
px.scatter(most_expensive, y="User Rating", x="Price", color="Genre",
                 size='Price')

Los 10 libros con mejor valoración

In [ ]:
total = 10
best_reviews = df_dedup.sort_values('User Rating', ascending=False)[:total]

fig = px.bar(best_reviews, y= 'User Rating', x='Name',  hover_data = best_reviews[['Price', 'Year','Genre']], color='Genre')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Los 10 libros con peor valoración

In [ ]:
total = 10
worst_reviews = df_dedup.sort_values('User Rating', ascending=True)[:total]

fig = px.bar(worst_reviews, y= 'User Rating', x='Name',  hover_data = worst_reviews[['Price', 'Year','Genre']], color='Genre')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Los 20 libros con más valoraciones

In [ ]:
total = 20
top_reviews = df_dedup.sort_values('Reviews', ascending=False)[:total]
fig = px.bar(top_reviews, y= 'Reviews', x='Name',  hover_data = top_reviews[['Price', 'Year','Genre']], color='Genre')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Los 20 libros con menos valoraciones

In [ ]:
total = 20
peores_valoraciones = df_dedup.sort_values('Reviews', ascending=True)[:total]
fig = px.bar(peores_valoraciones, x='Name', y= 'Reviews',  hover_data = peores_valoraciones[['Price', 'Year','Genre']], color='Genre')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Top 10 autores con más libros bestsellers

In [ ]:
total = 10
top_autores = df_dedup['Author'].value_counts()[:total]

fig = px.bar(top_autores, y=top_autores.values, x=top_autores.index, title='Top 10 autores',labels={'y':'Numero de libros', 'index':'Autor'})
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Top 10 autores de acuerdo al rating

In [ ]:
top_ratings_autores = df_dedup[df_dedup['Author'].isin(top_autores.index)][['Author','User Rating', 'Reviews']]
top_authors_ratings_agrupados=top_ratings_autores.groupby('Author')[['User Rating','Reviews']].mean().sort_values('Reviews', ascending=False)

fig = px.bar(top_authors_ratings_agrupados, y='User Rating', x= top_authors_ratings_agrupados.index, title='Top 10 autores con puntuación media')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()


In [ ]:
top_authors_ratings_agrupados=top_ratings_autores.groupby('Author')[['User Rating', 'Reviews']].mean().sort_values('Reviews', ascending=False)
fig = px.bar(top_authors_ratings_agrupados, x= top_authors_ratings_agrupados.index, y='Reviews', title='Top 10 autores con reseñas')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Autores con más reseñas de promedio y su ratio

In [ ]:
mas_rev_medias = df_dedup.groupby('Author').mean().sort_values('Reviews',ascending=False).reset_index()[:10]

fig = px.scatter(mas_rev_medias, x='User Rating', y='Reviews', color='Author')

fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

Autores con precios más altos de promedio y su ratio

In [ ]:
mas_caro_autor = df_dedup.groupby('Author').mean().sort_values('Price',ascending=False).reset_index()[:10]

fig = px.scatter(mas_caro_autor, x='User Rating', y='Price', color='Author')

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

In [ ]:
!pip install bubbly

evolución de las reseñas y puntuación a lo largo del tiempo

In [ ]:
from bubbly.bubbly import bubbleplot
from plotly.offline import iplot

genre_year = df_dedup.groupby(['Genre','Year']).mean().reset_index()
figure = bubbleplot(dataset=genre_year, x_column='User Rating', y_column='Reviews', 
    bubble_column='Genre', time_column='Year', color_column='Genre', 
    x_title="Promedio de puntuación del usuario", y_title="Promedio de reseñas", title='Reseñas vs Puntuación',
    x_logscale=True, scale_bubble=3, height=700)

iplot(figure)

evolución de la puntuación y reseñas a lo largo del tiempo seún el género

In [ ]:
p_res = df_dedup.groupby(['Genre','Year']).mean().reset_index()
figure = bubbleplot(dataset=p_res, x_column='User Rating', y_column='Price', 
    bubble_column='Genre', time_column='Year', color_column='Genre', 
    y_title="Promedio de puntuación del usuario", x_title="Promedio de reseñas", title='Puntuación vs. Reseñas',
    x_logscale=True, scale_bubble=3, height=650)

iplot(figure)

Autores con bestsellers gratis

In [ ]:
free_bestsellers = df_dedup[df_dedup["Price"]==0]

total = 10
top_autores_gratis = free_bestsellers['Author'].value_counts()[:total]

fig = px.bar(top_autores_gratis, x=top_autores_gratis.index, y=top_autores_gratis.values, title='Top 10 autores de bestsellers gratuitos',labels={'y':'Number of Books', 'index':'Author'})
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

Bestsellers gratuitos con mejor valoración

In [ ]:
top_9 = free_bestsellers.sort_values('User Rating', ascending=False)[:9]

fig = px.bar(top_9, x='Name', y= 'User Rating',  hover_data = top_9[['Year','Genre', 'Price']], color='Genre')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
top_9_reviews = free_bestsellers.sort_values('Reviews', ascending=False)[:9]
fig = px.bar(top_9_reviews, x='Name', y= 'Reviews',  hover_data = top_9_reviews[['Year','Genre', 'Price']], color='Genre')
fig.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()